In [1]:
import numpy as np

import plotly
import plotly.express as px
import plotly.graph_objects as go


from plot_funcs import (get_results, get_mean_avg_ranks, get_ranks_sizes,
                        get_highest_performing_df, get_cut_off_df,
                        get_across_ranks, get_intra_pipeline_df,
                        get_single_vs_multiple_df, get_results_df)

In [2]:
results = get_results('../exp/results')

static = {'random': True, 'base': True, 'fs': True, 'ico': True}
params = dict(results=results, keep_full_name=True, threshold=False, **static)

ts = [True, True, True, True]
fs = [False, False, False, False]

Found: 64530 Incomplete: 0


In [3]:
def add_base_side_annot(fig):
    
    # Add dropdown
    fig.update_layout(
        updatemenus=[
            dict(
                type = "buttons",
                direction = "left",
                buttons=buttons,
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.11,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    # Add annotation
    fig.update_layout(
        annotations=[
            dict(text="Scale:", showarrow=False,
                 x=0, y=1.08, yref="paper", align="left")
        ]
    )

## Base single parcellation with log toggle

In [ ]:
          text=df["text"],
               
               ))
            


,Parcellation_Type,Mean_Rank,Size,full_name,link
0,Existing,143.837037,83,aal_abox,"<a title=""Something"" href=""https://www.google...."
1,Existing,81.214815,343,aicha,"<a title=""Something"" href=""https://www.google...."
2,Existing,121.251852,172,baldassano_abox,"<a title=""Something"" href=""https://www.google...."
3,Existing,129.829630,85,brodmann,"<a title=""Something"" href=""https://www.google...."
4,Existing,147.466667,71,desikan_abox,"<a title=""Something"" href=""https://www.google...."
...,...,...,...,...,...
215,Existing,92.888889,70,vol-resamp-smith-bm70_prob,"<a title=""Something"" href=""https://www.google...."
216,Existing,86.125926,70,vol-resamp-smith-rsn70_prob,"<a title=""Something"" href=""https://www.google...."
217,Existing,172.881481,17,yeo_17networks,"<a title=""Something"" href=""https://www.google...."
218,Existing,190.096296,7,yeo_7networks,"<a title=""Something"" href=""https://www.google...."


In [68]:


# Init figure
fig = go.Figure()
fig.layout.xaxis.title = 'Size'
fig.layout.yaxis.title = 'Mean Rank'

def add_traces(r_df, visible=False):

    # By Parcellation type
    for name, df in r_df.groupby('Parcellation_Type'):
        
        gs = go.Scatter(
                x=df['Size'],
                y=df['Mean_Rank'],
                visible=visible,
                mode='markers+text',
                name=name,
                text=df['link'],
                textfont_size=8,
                hovertext=df['full_name'],
                textposition="middle center",
                hovertemplate='<b>%{hovertext}</b><br>' +
                              'Mean Rank: %{y:.3f}<br>' +
                              'Size: %{x:.3f}')
        
        gs.marker.size = 20
        gs.marker.opacity = .5

        # Add plot
        fig.add_trace(gs)
        
# Keep track of buttons
buttons = []

df = get_ranks_sizes(**params, log=False)
df['link'] = "<a href=\"https://www.google.com\">.</a>"

# Base
add_traces(df, visible=True)
#add_traces(get_ranks_sizes(**params, log=False), visible=True)
buttons.append(dict(args=[{"visible": ts + fs}],
                    label='Base',
                    method="update"))

# Log10
add_traces(df, visible=False)
#add_traces(get_ranks_sizes(**params, log=True), visible=False)
buttons.append(dict(args=[{"visible": fs + ts}],
                    label='Log',
                    method="update"))

# Add base side dropdown + annot
add_base_side_annot(fig)




plotly.offline.plot(fig)
#fig.write_html('../docs/_includes/interactive1.html')
#fig.write_html('../docs/interactive1.html')

'temp-plot.html'

## By Pipeline

In [5]:
models = ['LGBM', 'SVM', 'Elastic-Net', 'All']
p_types = get_ranks_sizes(**params)['Parcellation_Type'].unique()

fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

fig_dict["layout"]["xaxis"] = {"range": [.5, 4], "title": "Size"}
fig_dict["layout"]["yaxis"] = {"range": (1.25, 2.5), "title": "Mean Rank"}
fig_dict["layout"]["hovermode"] = "closest"


sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Pipeline:",
        "visible": True,
        "xanchor": "right"
    },
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

def get_data_dicts(pipeline):
    
    if pipeline == 'All':
        models = ['elastic', 'lgbm', 'svm']
    elif pipeline == 'Elastic-Net':
        models = ['elastic']
    elif pipeline == 'LGBM':
        models = ['lgbm']
    else:
        models = ['svm']
    
    r_df = get_ranks_sizes(**params, log=True, models=models)
    
    data_dicts = []
    for name, df in r_df.groupby('Parcellation_Type'):
        data_dict = {
            'x': df['Size'],
            'y': df['Mean_Rank'],
            'mode': "markers",
            'text': df['full_name'],
            "marker": {
                "size": 10,
                "opacity": .5
            },
            'name': name,
            'hovertemplate': '<b>%{text}</b><br>' +
                             'Mean Rank: %{y:.3f}<br>' +
                             'Size: %{x:.3f}'
        }
        data_dicts.append(data_dict)
        
    return data_dicts
            
# make init data
fig_dict["data"] = get_data_dicts(pipeline='LGBM')

# make frames
for model in models:
    frame = {"data": [], "name": str(model)}
    
    frame['data'] = get_data_dicts(model)
    fig_dict["frames"].append(frame)
    
    slider_step = {"args": [
        [model],
        {"frame": {"duration": 1000, "redraw": False},
         "transition": {"duration": 1000}}
    ],
        "label": model,
        "method": "animate"}
    
    sliders_dict["steps"].append(slider_step)
    
fig_dict["layout"]["sliders"] = [sliders_dict]
fig = go.Figure(fig_dict)

#plotly.offline.plot(fig)
fig.write_html('../docs/_includes/interactive2.html')
fig.write_html('../docs/interactive2.html')

## By Target - Avg Over Pipeline

In [6]:
# Init figure
fig = go.Figure()
fig.layout.xaxis.title = 'Size'
fig.layout.yaxis.title = 'Mean Rank'

def add_traces(r_df, visible=False):

    # By Parcellation type
    for name, df in r_df.groupby('target'):
        
        gs = go.Scatter(
                x=df['Size'],
                y=df['Mean_Rank'],
                visible=visible,
                mode='markers',
                name=name,
                text=df['full_name'],
                hovertemplate='<b>%{text}</b><br>' +
                              'Mean Rank: %{y:.3f}<br>' +
                              'Size: %{x:.3f}')
        
        gs.marker.size = 15
        gs.marker.opacity = .25

        # Add plot
        fig.add_trace(gs)

# Keep track of buttons
buttons = []

# Base
add_traces(get_ranks_sizes(avg_targets=False, log=False, **params), visible=True)
buttons.append(dict(args=[{"visible": ([True]*45) + ([False]*45)}],
                    label='Base',
                    method="update"))

# Log10
add_traces(get_ranks_sizes(avg_targets=False, log=True, **params), visible=False)
buttons.append(dict(args=[{"visible": ([False]*45) + ([True]*45)}],
                    label='Log',
                    method="update"))

# Add base side dropdown + annot
add_base_side_annot(fig)

#plotly.offline.plot(fig)
fig.write_html('../docs/_includes/interactive3.html')
fig.write_html('../docs/interactive3.html')

In [33]:
go.Scatter?